In [108]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'/home/ubuntu/varios/skforecast'

In [109]:
import pandas as pd
import numpy as np
from skforecast.datasets import fetch_dataset
from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
from skforecast.model_selection_multiseries import backtesting_forecaster_multiseries
from skforecast.model_selection_multiseries import grid_search_forecaster_multiseries
from skforecast.model_selection_multiseries import bayesian_search_forecaster_multiseries
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from skforecast.metrics import mean_absolute_scaled_error

In [110]:
import pandas as pd
import numpy as np
from skforecast.datasets import fetch_dataset
from skforecast.model_selection_multiseries.model_selection_multiseries import _calculate_metrics_multiseries
from skforecast.metrics import add_y_train_argument
from sklearn.metrics import mean_absolute_error
from skforecast.metrics import mean_absolute_scaled_error

In [111]:
data = pd.DataFrame(
    data={
        "item_1": [
            8.253175, 22.777826, 27.549099, 25.895533, 21.379238, 21.106643,
            20.533871, 20.069327, 20.006161, 21.620184, 21.717691, 21.751748,
            21.758617, 20.784194, 18.976196, 20.228468, 26.636444, 29.245869,
            24.772249, 24.018768, 22.503533, 20.794986, 23.981037, 28.018830,
            28.747482, 23.908368, 21.423930, 24.786455, 24.615778, 27.388275,
            25.724191, 22.825491, 23.066582, 23.788066, 23.360304, 23.119966,
            21.763739, 23.008517, 22.861086, 22.807790, 23.424717, 22.208947,
            19.558775, 20.788390, 23.619240, 25.061150, 27.646380, 25.609772,
            22.504042, 20.838095
        ],
        "item_2": [
            21.047727, 26.578125, 31.751042, 24.567708, 18.191667, 17.812500,
            19.510417, 24.098958, 20.223958, 19.161458, 16.042708, 14.815625,
            17.031250, 17.009375, 17.096875, 19.255208, 28.060417, 28.779167,
            19.265625, 19.178125, 19.688542, 21.690625, 25.332292, 26.675000,
            26.611458, 19.759375, 20.038542, 24.680208, 25.032292, 28.111458,
            21.542708, 16.605208, 18.593750, 20.667708, 21.977083, 29.040625,
            18.979167, 18.459375, 17.295833, 17.282292, 20.844792, 19.858333,
            18.446875, 19.239583, 19.903125, 22.970833, 28.195833, 20.221875,
            19.176042, 21.991667
        ],
        "item_3": [
            19.429739, 28.009863, 32.078922, 27.252276, 20.357737, 19.879148,
            18.043499, 26.287368, 16.315997, 21.772584, 18.729748, 12.552534,
            18.996209, 18.534327, 15.418361, 16.304852, 30.076258, 28.886334,
            20.286651, 21.367727, 20.248170, 19.799975, 25.931558, 27.698196,
            30.725005, 19.573577, 23.310162, 24.959233, 24.399246, 29.094136,
            22.639513, 18.372362, 21.256450, 22.430527, 19.575067, 31.767626,
            20.086271, 21.380186, 17.553807, 17.369879, 21.829746, 16.208510,
            25.067215, 21.863615, 17.887458, 23.005424, 25.013939, 22.142083,
            23.673005, 25.238480
        ],
    },
    index=pd.date_range(start="2012-01-01", end="2012-02-19"),
)


predictions_different_lenght = pd.DataFrame(
    data={
        "item_1": [
            25.849411, 24.507137, 23.885447, 23.597504, 23.464140, 23.402371,
            23.373762, 23.360511, 23.354374, 23.351532, 23.354278, 23.351487,
            23.350195, 23.349596, 23.349319, 23.349190, 23.349131, 23.349103,
            23.349090, 23.349084, 23.474207, 23.407034, 23.375922, 23.361512,
            23.354837
        ],
        "item_2": [
            24.561460, 23.611980, 23.172218, 22.968536, 22.874199, 22.830506,
            22.810269, 22.800896, 22.796555, 22.794544, 22.414996, 22.617821,
            22.711761, 22.755271, 22.775423, 22.784756, 22.789079, 22.791082,
            22.792009, 22.792439, 21.454419, 22.172918, 22.505700, 22.659831,
            22.731219
        ],
        "item_3": [
            26.168069, 24.057472, 23.079925, 22.627163, 22.417461, 22.320335,
            22.275350, 22.254515, 22.244865, 22.240395, 21.003848, 21.665604,
            np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan,
            np.nan, np.nan, np.nan, np.nan, np.nan
        ],
    },
    index=pd.date_range(start="2012-01-26", periods=25)
)


span_index = span_index = pd.date_range(start="2012-01-01", end="2012-02-19", freq="D")

folds = [
    [[0, 25], [24, 25], [25, 35], [25, 35], False],
    [[0, 25], [34, 35], [35, 45], [35, 45], False],
    [[0, 25], [44, 45], [45, 50], [45, 50], False],
]

levels = ["item_1", "item_2", "item_3"]

In [112]:
def test_calculate_metrics_multiseries_output_when_aggregated_metric_and_predictions_have_different_length(
    metrics=[mean_absolute_error, mean_absolute_scaled_error]
):

    metrics = [add_y_train_argument(metric) for metric in metrics]
    results = _calculate_metrics_multiseries(
        series=data,
        predictions=predictions_different_lenght,
        folds=folds,
        span_index=span_index,
        metrics=metrics,
        levels=levels,
        add_aggregated_metric=True,
    )

    expected = pd.DataFrame(
        data={
            "levels": [
                "item_1",
                "item_2",
                "item_3",
                "average",
                "weighted_average",
                "pooling",
            ],
            "mean_absolute_error": [
                1.477567,
                3.480129,
                3.173683,
                2.710460,
                2.613332,
                2.613332,
            ],
            "mean_absolute_scaled_error": [
                0.610914,
                1.170113,
                0.707757,
                0.829595,
                0.855141,
                0.793768,
            ],
        }
    )
    display(results)
    pd.testing.assert_frame_equal(results, expected)


test_calculate_metrics_multiseries_output_when_aggregated_metric_and_predictions_have_different_length()

,levels,mean_absolute_error,mean_absolute_scaled_error
0,item_1,1.477567,0.610914
1,item_2,3.480129,1.170113
2,item_3,3.173683,0.707757
3,average,2.710460,0.829595
4,weighted_average,2.613332,0.855141
5,pooling,2.613332,0.793768


In [113]:
(data.iloc[25:] - predictions).abs().stack().mean()

nan

In [114]:
np.average([1.477567, 3.480129, 3.173683], weights=[25, 25, 12])

2.613332193548387

In [115]:
predictions.notna().sum()

id_1000    153
id_1001    153
id_1003     80
id_1004     31
dtype: int64

In [116]:
# Unit test backtesting_forecaster_multiseries
# ==============================================================================
import re
import pytest
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

from skforecast.exceptions import IgnoredArgumentWarning
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
from skforecast.ForecasterAutoregMultiSeriesCustom import ForecasterAutoregMultiSeriesCustom
from skforecast.ForecasterAutoregMultiVariate import ForecasterAutoregMultiVariate
from skforecast.model_selection_multiseries import backtesting_forecaster_multiseries
from skforecast.model_selection_multiseries import backtesting_forecaster_multivariate

# Fixtures
from skforecast.model_selection_multiseries.tests.fixtures_model_selection_multiseries import series
THIS_DIR = Path("/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/tests")
series_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series.joblib')
exog_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series_exog.joblib')
end_train = "2016-07-31 23:59:00"
series_dict_train = {k: v.loc[:end_train,] for k, v in series_dict.items()}
exog_dict_train = {k: v.loc[:end_train,] for k, v in exog_dict.items()}
series_dict_test = {k: v.loc[end_train:,] for k, v in series_dict.items()}
exog_dict_test = {k: v.loc[end_train:,] for k, v in exog_dict.items()}
series_with_nans = series.copy()
series_with_nans.iloc[:10, series_with_nans.columns.get_loc('l2')] = np.nan

In [117]:
@pytest.mark.parametrize("forecaster", 
    [ForecasterAutoregMultiSeries(
        regressor=LGBMRegressor(
            n_estimators=2, random_state=123, verbose=-1, max_depth=2
        ),
        lags=14,
        encoding='ordinal',
        dropna_from_series=False,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler(),
    ), 
    ForecasterAutoregMultiSeriesCustom(
        regressor=LGBMRegressor(
            n_estimators=2, random_state=123, verbose=-1, max_depth=2
        ),
        fun_predictors=create_predictors_14, 
        window_size=14,
        encoding='ordinal',
        dropna_from_series=False,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler(),
    )], 
    ids=lambda forecaster: f'forecaster: {type(forecaster).__name__}')

SyntaxError: incomplete input (2598295151.py, line 23)

In [ ]:
forecaster = ForecasterAutoregMultiSeries(
        regressor=LGBMRegressor(
            n_estimators=2, random_state=123, verbose=-1, max_depth=2
        ),
        lags=14,
        encoding='ordinal',
        dropna_from_series=False,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler(),
    )


def test_output_backtesting_forecaster_multiseries_ForecasterAutoregMultiSeries_series_and_exog_dict_with_mocked_multiple_aggregated_metrics():
    """
    Test output of backtesting_forecaster_multiseries in ForecasterAutoregMultiSeries 
    and ForecasterAutoregMultiSeriesCustom when series and exog are
    dictionaries and multiple aggregated metrics are calculated.
    (mocked done in Skforecast v0.12.0).
    """
    
    metrics, predictions = backtesting_forecaster_multiseries(
        forecaster            = forecaster,
        series                = series_dict,
        exog                  = exog_dict,
        steps                 = 24,
        metric                = ['mean_absolute_error', 'mean_absolute_scaled_error'],
        add_aggregated_metric = True,
        initial_train_size    = len(series_dict_train['id_1000']),
        fixed_train_size      = True,
        gap                   = 0,
        allow_incomplete_fold = True,
        refit                 = False,
        n_jobs                = 'auto',
        verbose               = False,
        show_progress         = False,
        suppress_warnings     = True
    )

    expected_metrics = pd.DataFrame(
        data=np.array([['id_1000', 286.6227398656757, 105816.86051259708],
        ['id_1001', 1364.7345740769094, 2175934.9583102698],
        ['id_1003', 237.4894217124842, 95856.72602398091],
        ['id_1004', 1267.85941538558, 2269796.338792736],
        ['average', 789.1765377601623, 1161851.2209098958],
        ['weighted_average', 745.7085483145497, 1024317.153152019],
        ['pooling', 745.7085483145497, 1024317.1531520189]], dtype=object),
        columns=['levels', 'mean_absolute_error', 'mean_squared_error']
    ).astype({'mean_absolute_error': float, 'mean_squared_error': float})
    
    expected_predictions = pd.DataFrame(
    data=np.array([[1438.14154717, 2090.79352613, 2166.9832933, 7285.52781428],
                   [1438.14154717, 2089.11038884, 2074.55994929, 7488.18398744],
                   [1438.14154717, 2089.11038884, 2035.99448247, 7488.18398744],
                   [1403.93625654, 2089.11038884, 2035.99448247, 7488.18398744],
                   [1403.93625654, 2089.11038884, 2035.99448247, 7488.18398744],
                   [1403.93625654, 2076.10228838, 2035.99448247, 7250.69119259],
                   [1403.93625654, 2076.10228838, np.nan, 7085.32315355],
                   [1403.93625654, 2000.42985714, np.nan, 7285.52781428],
                   [1403.93625654, 2013.4379576, np.nan, 7285.52781428],
                   [1403.93625654, 2013.4379576, np.nan, 7285.52781428]]),
    columns=['id_1000', 'id_1001', 'id_1003', 'id_1004'],
    index=pd.date_range('2016-08-01', periods=10, freq='D')
)

    pd.testing.assert_frame_equal(metrics, expected_metrics)
    pd.testing.assert_frame_equal(predictions.head(10), expected_predictions)

test_output_backtesting_forecaster_multiseries_ForecasterAutoregMultiSeries_series_and_exog_dict_with_mocked_multiple_aggregated_metrics()

AssertionError: DataFrame.columns are different

DataFrame.columns values are different (33.33333 %)
[left]:  Index(['levels', 'mean_absolute_error', 'mean_absolute_scaled_error'], dtype='object')
[right]: Index(['levels', 'mean_absolute_error', 'mean_squared_error'], dtype='object')
At positional index 2, first diff: mean_absolute_scaled_error != mean_squared_error

In [118]:
metrics, predictions = backtesting_forecaster_multiseries(
        forecaster            = forecaster,
        series                = series_dict,
        exog                  = exog_dict,
        steps                 = 24,
        metric                = ['mean_absolute_error', 'mean_absolute_scaled_error'],
        add_aggregated_metric = True,
        initial_train_size    = len(series_dict_train['id_1000']),
        fixed_train_size      = True,
        gap                   = 0,
        allow_incomplete_fold = True,
        refit                 = False,
        n_jobs                = 'auto',
        verbose               = False,
        show_progress         = False,
        suppress_warnings     = True
    )

predictions.head(10)

,id_1000,id_1001,id_1003,id_1004
2016-08-01,1438.141547,2090.793526,2166.983293,7285.527814
2016-08-02,1438.141547,2089.110389,2074.559949,7488.183987
2016-08-03,1438.141547,2089.110389,2035.994482,7488.183987
2016-08-04,1403.936257,2089.110389,2035.994482,7488.183987
2016-08-05,1403.936257,2089.110389,2035.994482,7488.183987
2016-08-06,1403.936257,2076.102288,2035.994482,7250.691193
2016-08-07,1403.936257,2076.102288,NaN,7085.323154
2016-08-08,1403.936257,2000.429857,NaN,7285.527814
2016-08-09,1403.936257,2013.437958,NaN,7285.527814
2016-08-10,1403.936257,2013.437958,NaN,7285.527814


In [124]:
metrics

,levels,mean_absolute_error,mean_absolute_scaled_error
0,id_1000,286.622740,1.301728
1,id_1001,1364.734574,3.878227
2,id_1003,237.489422,0.994727
3,id_1004,1267.859415,1.060634
4,average,789.176538,1.808829
5,weighted_average,745.708548,2.170242
6,pooling,745.708548,1.771577


In [107]:
y_true = series_dict_test['id_1003'].loc[predictions.index]
y_pred = predictions['id_1003']
y_train = series_dict_train['id_1003']

if not isinstance(y_true, (pd.Series, np.ndarray)):
    raise TypeError("y_true must be a pandas Series or numpy array")
if not isinstance(y_pred, (pd.Series, np.ndarray)):
    raise TypeError("y_pred must be a pandas Series or numpy array")
if not isinstance(y_train, (list, pd.Series, np.ndarray)):
    raise TypeError("y_train must be a list, pandas Series or numpy array")
if isinstance(y_train, list):
    for x in y_train:
        if not isinstance(x, (pd.Series, np.ndarray)):
            raise TypeError(
                "When `y_train` is a list, each element must be a pandas Series "
                "or numpy array"
            )
if len(y_true) != len(y_pred):
    raise ValueError("y_true and y_pred must have the same length")
if len(y_true) == 0 or len(y_pred) == 0:
    raise ValueError("y_true and y_pred must have at least one element")

if isinstance(y_train, list):
    naive_forecast = np.concatenate([np.diff(x) for x in y_train])
    mase = np.mean(np.abs(y_true - y_pred)) / np.mean(np.abs(naive_forecast))

else:
    mase = np.mean(np.abs(y_true - y_pred)) / np.nanmean(np.abs(np.diff(y_train)))

mase

0.9947270993025895

In [98]:
mean_absolute_scaled_error(series_dict_test['id_1003'].loc[predictions.index], predictions['id_1003'], series_dict_train['id_1003'].dropna())

0.9946632167597738

In [123]:
series_dict_train['id_1003']

timestamp
2016-01-01    2294.750893
2016-01-02    1750.000198
2016-01-03    1455.750492
2016-01-04    2141.000198
2016-01-05    2384.870697
                 ...     
2016-07-27    1830.871780
2016-07-28    1776.250992
2016-07-29    1644.001091
2016-07-30    1943.500587
2016-07-31    3658.369995
Freq: D, Name: id_1003, Length: 213, dtype: float64